In [1]:
import dataextraction as de
from bs4 import BeautifulSoup
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re,string,unicodedata
from string import punctuation
import numpy as np
import pandas as pd
%matplotlib inline
%load_ext autoreload
%autoreload 2

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/rohangoel66/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
print(torch.cuda.get_device_name(0))

cuda:0
Tesla K80


In [3]:
data = de.extract_data('data/train.jsonl')
train = de.parse_json(data, True)

data = de.extract_data('data/test.jsonl')
test = de.parse_json(data, False)

In [4]:
print(len(train.index))
print(len(test.index))

5000
1800


In [5]:
stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)

In [6]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

def remove_between_square_brackets(text):
    return re.sub(r'http\S+', '', text)

def remove_stopwords(text):
    final_text = []
    for i in text.split():
        if i.strip().lower() not in stop and i != "@USER":
            final_text.append(i.strip().lower())
    return " ".join(final_text)

def denoise_text(text):
    text = strip_html(text)
    text = remove_between_square_brackets(text)
    text = remove_stopwords(text)
    return text

train['response']=train['response'].apply(denoise_text)
test['response']=test['response'].apply(denoise_text)

In [7]:
train.head()

,label,response
0,SARCASM,get .. obviously care would've moved right alo...
0,SARCASM,trying protest talking labels label wtf make em
0,SARCASM,makes insane money movies einstein #learnhowth...
0,SARCASM,meanwhile trump even release sat scores wharto...
0,SARCASM,pretty sure anti-lincoln crowd claimed democra...


In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

In [9]:
train = train.replace("SARCASM", 1)
train = train.replace("NOT_SARCASM", 0)

In [10]:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [11]:
tokenized = train['response'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, max_length = 512, truncation = True)))

In [12]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [13]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(5000, 73)

In [14]:
input_ids = torch.tensor(padded)
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [15]:
train_features = last_hidden_states[0][:,0,:].numpy()
train_labels = train['label']

In [16]:
tokenized = test['response'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [17]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [18]:
attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

(1800, 49)

In [19]:
input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [20]:
lr_clf = LogisticRegression(max_iter = 10000)
lr_clf.fit(train_features, train_labels)

LogisticRegression(max_iter=10000)

In [21]:
lr_clf.score(train_features, train_labels)

0.8122

In [22]:
test_features = last_hidden_states[0][:,0,:].numpy()
test_pred = lr_clf.predict(test_features)

In [23]:
from numpy import savetxt
savetxt('train_features.csv', train_features, delimiter=',')
savetxt('train_labelst.csv', train_labels.to_numpy(), delimiter=',')
savetxt('test_features.csv', test_features, delimiter=',')

In [24]:
print(train_features.shape[1])
print(test_features.shape[1])

768
768


In [ ]:
from numpy import loadtxt
loadtxt('train_features.csv', delimiter=',')
loadtxt('train_labels.csv', delimiter=',')
loadtxt('test_featurest.csv', delimiter=',')

In [38]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self, n_features):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(n_features, 256)
        self.fc2 = nn.Linear(256, 80)
        self.fc3 = nn.Linear(80, 1)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return torch.sigmoid(self.fc3(x))


net = Net(train_features.shape[1])

In [39]:
params = list(net.parameters())

In [40]:
X_train = torch.from_numpy(train_features).float()
y_train = torch.squeeze(torch.from_numpy(train_labels.to_numpy()).float())
X_test = torch.from_numpy(test_features).float()

In [41]:
from sklearn.model_selection import train_test_split
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=RANDOM_SEED)

In [42]:
criterion = nn.BCELoss()

In [43]:
# optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay = 0.05)
optimizer = optim.AdamW(params, lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01, amsgrad=False)

In [44]:
device = torch.device("cpu")
X_train = X_train.to(device)
y_train = y_train.to(device)
X_val = X_val.to(device)
y_val = y_val.to(device)
X_test = X_test.to(device)
net = net.to(device)
criterion = criterion.to(device)

In [45]:
def calculate_accuracy(y_true, y_pred):
    predicted = y_pred.ge(.5).view(-1)
    return (y_true == predicted).sum().float() / len(y_true)

In [46]:
from sklearn.metrics import f1_score
epochs = 500
def round_tensor(t, decimal_places=3):
    return round(t.item(), decimal_places)
for epoch in range(1, epochs + 1):
    y_pred = net(X_train)
    y_pred = torch.squeeze(y_pred)
    train_loss = criterion(y_pred, y_train)
    if epoch % (epochs / 10) == 0:
        predicted = y_pred.ge(.5).view(-1)
        train_acc = f1_score(predicted, y_train, average="macro")
        y_val_pred = net(X_val)
        y_val_pred = torch.squeeze(y_val_pred)
        val_loss = criterion(y_val_pred, y_val)
        val_predicted = y_val_pred.ge(.5).view(-1)
        val_acc = f1_score(val_predicted, y_val, average="macro")
        print(f'''epoch {epoch} Train set - loss: {round_tensor(train_loss)}, accuracy: {round_tensor(train_acc)}
        Val  set - loss: {round_tensor(val_loss)}, accuracy: {round_tensor(val_acc)}''')
    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

ValueError: Samplewise metrics are not available outside of multilabel classification.

In [ ]:
y_test = net(X_test)
print(y_test)
y_labels = []
for i in range(y_test.shape[0]):
    if y_test[i] < 0.5:
        y_labels.append("NOT_SARCASM")
    else:
        y_labels.append("SARCASM")

In [ ]:
with open('answer.txt', 'w') as f:
    for tid, pred in zip(test['id'], y_labels):
        f.writelines(f'{tid},{pred}\n')